In [3]:
from pandas_datareader import data
import datetime

In [5]:
start=datetime.datetime(2022,5,16)
end=datetime.datetime(2022,5,28)
df = data.DataReader(name= "AAPL", data_source="yahoo", start=start, end=end)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-05-16,147.520004,144.179993,145.550003,145.539993,86643800,145.539993
2022-05-17,149.770004,146.679993,148.860001,149.240005,78336300,149.240005
2022-05-18,147.360001,139.899994,146.850006,140.820007,109742900,140.820007
2022-05-19,141.660004,136.600006,139.880005,137.350006,136095600,137.350006
2022-05-20,140.699997,132.610001,139.089996,137.589996,137194600,137.589996
